In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from glob import glob
from PIL import Image
from functions import left, right, up, down, get_SC, get_filename

%matplotlib widget

In [ ]:
X, Y = np.loadtxt('coords.txt', unpack=True)
tot = len(X)
N = np.arange(tot)

image = Image.open(get_filename(tot//2))  
vmin = np.percentile(image, 1)*0.5
vmax = np.percentile(image, 99)*2

def update(n=0, vmin=vmax, vmax=vmax, Autov=False):
    
    image = Image.open(get_filename(n))
    art_image.set_array(image)
    axs[0].set_title(f'n={n}')

    if Autov:
        vmin = np.percentile(image, 5)
        print(vmin)
        vmax = np.percentile(image, 99.9)
        vminslider.min = vmin
        vminslider.max = vmax*1.5
        vmaxslider.min = vmin
        vmaxslider.max = vmax*1.5
        
    art_image.set_clim(vmin, vmax)

    S, C = get_SC(n, X, Y)
    art_points.set_sizes(S)
    art_points.set_color(C)
    axs[1].set_title(f'X={X[n]:.3f}, Y={Y[n]:.3f}')

In [ ]:
with plt.ioff():
    fig, axs = plt.subplots(1, 2, figsize=(9, 3), width_ratios=[2, 1])
    
    n = 0
    
    image = Image.open(get_filename(n))
    
    art_image = axs[0].imshow(image, cmap='Greys_r',
                              vmin=vmin, vmax=vmax)
    axs[0].set_title(f'n={n}')
    
    S, C = get_SC(n, X, Y)
    art_points = axs[1].scatter(X, Y, s=S, c=C)
    axs[1].set_aspect((X.max() - X.min())/(Y.max() - Y.min()))
    axs[1].set_title(f'n={n}')
    axs[1].set_xlabel('X')
    axs[1].set_ylabel('Y')

n = 0
for x, y in zip(X, Y):
    axs[1].text(x, y, f'{n}', fontsize=7, horizontalalignment='left', verticalalignment='bottom')
    n += 1

# Define the button labels
buttons = ['↑', '↓', '→', '←']

# Create buttons for each label
button_widgets = {label: ipywidgets.Button(description=label, layout=ipywidgets.Layout(width='25px', height='25px'),
                                          fontsize=4)
                  for label in buttons}


button_grid = ipywidgets.GridspecLayout(3, 3, height='120px', width='120px')

# Place the buttons in the grid
button_grid[0, 1] = button_widgets['↑']
button_grid[2, 1] = button_widgets['↓']
button_grid[1, 2] = button_widgets['→']
button_grid[1, 0] = button_widgets['←']

nslider = ipywidgets.IntSlider(value=0, min=0, max=(tot - 1), step=1)
vminslider = ipywidgets.FloatSlider(value=vmin, min=vmin, max=vmax, step=vmax/100)
vmaxslider = ipywidgets.FloatSlider(value=vmax, min=vmin, max=vmax, step=vmax/100)
Autovslider = ipywidgets.Checkbox(value=False)

def update_n_up(_):
    nslider.value = up(nslider.value, N, X, Y)
button_widgets['↑'].on_click(update_n_up)

def update_n_down(_):
    # dval = 
    nslider.value = down(nslider.value, N, X, Y)
    # nslider.value = down(nslider.value, N, X, Y)
button_widgets['↓'].on_click(update_n_down)

def update_n_right(_):
    nslider.value = right(nslider.value, N, X, Y)
button_widgets['→'].on_click(update_n_right)

def update_n_left(_):
    nslider.value = left(nslider.value, N, X, Y)
button_widgets['←'].on_click(update_n_left)


image_widget = ipywidgets.interactive(update,
                                      n=nslider,
                                      vmin=vminslider,
                                      vmax=vmaxslider,
                                      Autov=Autovslider)

sliders = ipywidgets.VBox((nslider, vminslider, vmaxslider, Autovslider))
top = ipywidgets.HBox((sliders, button_grid))
display(top, fig.canvas)